# Tests sur les dimensions d'images

Le but de ce test est de déterminer la dimension idéale des images.

In [1]:
import os
import time

import cv2
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score

import albumentations as alb

import tensorflow as tf
from tensorflow.keras import layers

import tools

INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.10 (you have 1.4.8). Upgrade using: pip install --upgrade albumentations
2024-07-02 21:21:23.210767: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-02 21:21:23.211934: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-02 21:21:23.254742: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-02 21:21:23.391522: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in oth

In [ ]:
max_image_size = (600, 600)
x_total = []
y_total = []

def load_datasets(img, category):
    new_img = cv2.resize(img, max_image_size)
    x_total.append(new_img)
    category = 0 if category == "NORMAL" else 1
    y_total.append(category)


tools.browse_imgs(load_datasets)

steps = [(100, 100), (200, 200), (400, 400), max_image_size]
results = {}
for image_size in steps:
    print("img_size : ", image_size)
    
    num_classes = 1
    core_size = 8

    model = tf.keras.Sequential([
        layers.Input(shape=(*image_size, 1)),
        layers.Conv2D(16, core_size, activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(32, core_size, activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(64, core_size, activation='relu'),
        layers.MaxPooling2D(),

        layers.Flatten(),
        layers.Dense(100, activation='relu'),
        layers.Dense(200, activation='relu'),
        layers.Dense(300, activation='relu'),
        layers.Dense(num_classes, activation='sigmoid')
    ])

    metrics = ['recall', 'precision']
    model.compile(optimizer='adam',
                  loss=tf.losses.BinaryCrossentropy(),
                  metrics=metrics)
    
    x_train = []
    y_train = y_total
    
    for img in x_total:
        new_img = cv2.resize(img, image_size)
        x_train.append(new_img)        
        
    x_train = np.array(x_train) / 255
    y_train = np.array(y_train)
    
    x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.2)
    x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=0.5)
    
    start_time = time.time()
    model.fit(x_train,
              y_train,
              validation_data=(x_val, y_val),
              epochs=2,
              verbose=1)
    
    y_pred = model.predict(x_test, verbose=0)
    y_pred_label = np.round(y_pred).astype(int)
    results[image_size] = recall_score(y_test, y_pred_label)
    print("time of execution for ",image_size," size : ", time.time() - start_time)
    
    
plt.plot(results.keys(), results.values(), marker='.', label='Test Recall')
plt.xlabel('Image Size')
plt.ylabel('Model recall')
plt.title('validation Curve on image size')
plt.show()

img_size :  (100, 100)
Epoch 1/2
147/147 ━━━━━━━━━━━━━━━━━━━━ 7s 40ms/step - loss: 0.5758 - precision: 0.7426 - recall: 0.9912 - val_loss: 0.3489 - val_precision: 0.8330 - val_recall: 0.9529
Epoch 2/2
147/147 ━━━━━━━━━━━━━━━━━━━━ 6s 38ms/step - loss: 0.2865 - precision: 0.9124 - recall: 0.9218 - val_loss: 0.2311 - val_precision: 0.9490 - val_recall: 0.9231
time of execution for  (100, 100)  size :  13.555516004562378
img_size :  (200, 200)
Epoch 1/2
147/147 ━━━━━━━━━━━━━━━━━━━━ 31s 205ms/step - loss: 0.5655 - precision: 0.7544 - recall: 0.9793 - val_loss: 0.2853 - val_precision: 0.8600 - val_recall: 0.9840
Epoch 2/2
147/147 ━━━━━━━━━━━━━━━━━━━━ 30s 202ms/step - loss: 0.2128 - precision: 0.9422 - recall: 0.9442 - val_loss: 0.1776 - val_precision: 0.9292 - val_recall: 0.9908
time of execution for  (200, 200)  size :  63.54679322242737
img_size :  (400, 400)
